I'm going to start with a random initialization of CE with no weight, everything normal just to see what this performance is around. Going to go by validation auroc

Tensorboard results saved in subfolder called revised_tuning

This is the baseline for the 2 class multiclass 

CUDA_VISIBLE_DEVICES=3 python trainingLoop.py -lr 2e-4 -w 1.0,1.0 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 30 --simon_model --add_name 08-22-23 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass &

The AUROC best seems to be 0.85 on the val set and a 1F1 of 58%



Now trying the same parameters but with BCE and with a single class

CUDA_VISIBLE_DEVICES=3 python trainingLoop.py -lr 2e-4 -w 1.0,1.0 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 30 --simon_model --add_name 08-22-23_bce --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary &

Now trying the same parameters but with BCE and with a single class and PE Fixed

CUDA_VISIBLE_DEVICES=1 python trainingLoop.py -lr 2e-4 -w 1.0,1.0 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 30 --simon_model --add_name 08-22-23_bce_fixedpe --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_fixed &

Now trying the same parameters but with BCE and with a single class and PE Learned

CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 2e-4 -w 1.0,1.0 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 30 --simon_model --add_name 08-22-23_bce_learnedpe --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_learned &


Seems like PE Fixed is making it worse , learned PE barely any difference

Now going to try CE with fixed and learned PE

CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 2e-4 -w 1.0,1.0 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 30 --simon_model --add_name 08-22-23_fixedpe --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed &

CUDA_VISIBLE_DEVICES=3 python trainingLoop.py -lr 2e-4 -w 1.0,1.0 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 30 --simon_model --add_name 08-22-23_learnedpe --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_learned &

Fixed PE seems to have very slight improvement, but I'm going to keep it because I think it will make the attention make more sense in terms of how it is understanding the latent space 

Now going to take the fixed PE and add weighted CE just to see without two stage training what is the max performance we can expect

CUDA_VISIBLE_DEVICES=1 python trainingLoop.py -lr 2e-4 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 30 --simon_model --add_name 08-22-23_fixedpe --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed &

CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 2e-4 -w 1.0,5.0 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 30 --simon_model --add_name 08-22-23_fixedpe --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed &

Did not actually improve 1F1 much so going to ignore this for now, also did not help the MacroF1

Now for tuning: Going to try tuning just last layer, last 2 layers, last 3 layers

CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_TUNE-fc3 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_16_epochs_30_dpt_0.5_fold0_256,64,16_heads4_08-22-23_fixedpe_PEamp_1/lr_0.0002_w_1.0,1.0_bs_16_heads4_0.5_att08-22-23_fixedpe_epochs30_fold0.pt &

CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_TUNE-fc3fc2 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_16_epochs_30_dpt_0.5_fold0_256,64,16_heads4_08-22-23_fixedpe_PEamp_1/lr_0.0002_w_1.0,1.0_bs_16_heads4_0.5_att08-22-23_fixedpe_epochs30_fold0.pt &

CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_TUNE-fc3fc2fc1 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_16_epochs_30_dpt_0.5_fold0_256,64,16_heads4_08-22-23_fixedpe_PEamp_1/lr_0.0002_w_1.0,1.0_bs_16_heads4_0.5_att08-22-23_fixedpe_epochs30_fold0.pt &

Going to try with dropout of 0.5 as well to see if setting to 0 helps or hurts

CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_TUNE-fc3fc2 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_16_epochs_30_dpt_0.5_fold0_256,64,16_heads4_08-22-23_fixedpe_PEamp_1/lr_0.0002_w_1.0,1.0_bs_16_heads4_0.5_att08-22-23_fixedpe_epochs30_fold0.pt &

Now Going to take a step back and try to optimize the learning rate, num epochs, dropout, batch size

Used the random tuner, the following tuner seems like best performance:

CHOOSING THESE FOR STAGE 1 RESULTS

exp_lr_0.0006_w_1.0,1.0_ds_eeg_bs_16_epochs_16_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1   ## lowest val loss ## seemed to be worst tuned
* exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_16_epochs_30_dpt_0.5_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1   ## most balanced
* exp_lr_0.0006_w_1.0,1.0_ds_eeg_bs_16_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1	 ## best val auroc
exp_lr_0.002_w_1.0,1.0_ds_eeg_bs_10_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1    ## best 1F1 ## seemed to be worst tuned



Now I need to figure out whether to do 1,2 layer tuning, and CE or BCE tuning on each one. this is just normal CE tuning with 1 layer vs 2 layer:

CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE1-fc3 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.002_w_1.0,1.0_ds_eeg_bs_10_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.002_w_1.0,1.0_bs_10_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &

* CUDA_VISIBLE_DEVICES=1 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE2-fc3 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_16_epochs_30_dpt_0.5_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.0002_w_1.0,1.0_bs_16_heads4_0.5_attPARAM_TUNER_08-24-23_epochs30_fold0.pt &

CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE3-fc3 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0006_w_1.0,1.0_ds_eeg_bs_16_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.0006_w_1.0,1.0_bs_16_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &

CUDA_VISIBLE_DEVICES=3 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE4-fc3 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.002_w_1.0,1.0_ds_eeg_bs_10_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.002_w_1.0,1.0_bs_10_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &



CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE1-fc3-fc2 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.002_w_1.0,1.0_ds_eeg_bs_10_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.002_w_1.0,1.0_bs_10_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &

CUDA_VISIBLE_DEVICES=1 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE2-fc3-fc2 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_16_epochs_30_dpt_0.5_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.0002_w_1.0,1.0_bs_16_heads4_0.5_attPARAM_TUNER_08-24-23_epochs30_fold0.pt &

* CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE3-fc3-fc2 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0006_w_1.0,1.0_ds_eeg_bs_16_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.0006_w_1.0,1.0_bs_16_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &

CUDA_VISIBLE_DEVICES=3 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 2 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE4-fc3-fc2 --hidden_size 8 --fc2_size 32 --dropout 0.5 --task multiclass --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.002_w_1.0,1.0_ds_eeg_bs_10_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.002_w_1.0,1.0_bs_10_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &


Seemed like very slight difference between 1 and 2 layer tuning. Going to stick with TUNE3 2 layer tuning and TUNE2 1 layer tuning

Now going to do BCE with 1 and 2 layer for all four models

2 layer: 

CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE1-fc3_BCE --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.002_w_1.0,1.0_ds_eeg_bs_10_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.002_w_1.0,1.0_bs_10_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &

CUDA_VISIBLE_DEVICES=1 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE2-fc3_BCE --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_16_epochs_30_dpt_0.5_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.0002_w_1.0,1.0_bs_16_heads4_0.5_attPARAM_TUNER_08-24-23_epochs30_fold0.pt &

CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE3-fc3_BCE --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0006_w_1.0,1.0_ds_eeg_bs_16_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.0006_w_1.0,1.0_bs_16_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &

CUDA_VISIBLE_DEVICES=3 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE4-fc3_BCE --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.002_w_1.0,1.0_ds_eeg_bs_10_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.002_w_1.0,1.0_bs_10_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &



CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE1-fc3-fc2_BCE --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.002_w_1.0,1.0_ds_eeg_bs_10_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.002_w_1.0,1.0_bs_10_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &

CUDA_VISIBLE_DEVICES=1 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE2-fc3-fc2_BCE --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_16_epochs_30_dpt_0.5_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.0002_w_1.0,1.0_bs_16_heads4_0.5_attPARAM_TUNER_08-24-23_epochs30_fold0.pt &

CUDA_VISIBLE_DEVICES=2 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE3-fc3-fc2_BCE --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0006_w_1.0,1.0_ds_eeg_bs_16_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.0006_w_1.0,1.0_bs_16_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &

CUDA_VISIBLE_DEVICES=3 python trainingLoop.py -lr 0.002 -w 1.0,2.5 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE4-fc3-fc2_BCE --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.002_w_1.0,1.0_ds_eeg_bs_10_epochs_19_dpt_0.3_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.002_w_1.0,1.0_bs_10_heads4_0.3_attPARAM_TUNER_08-24-23_epochs19_fold0.pt &

potentially could also try adding the last layer and tuning the last two layers, but going to ignore this for now as it is technically unconventional 

TUNE 2 and TUNE 3 definitely performed the best meaning the val auroc is the most important metric to look at for the first phase of training

Out of personal preference, going to take the BCE results of two stage tuning because that is easiest to explain 

For the second stage results , if I choose based on best val auroc performance:
* exp_lr_0.002_w_1.0,2.5_ds_eeg_bs_16_epochs_15_dpt_0.0_fold0_256,64,16_heads4_08-22-23_fixedpe_PHASE2-TUNE3-fc3-fc2_BCE_PEamp_1/lr_0.002_w_1.0,2.5_bs_16_heads4_0.0_att08-22-23_fixedpe_PHASE2-TUNE3-fc3-fc2_BCE_epochs15_fold0.pt

If I chose based on F1 score:
* exp_lr_0.002_w_1.0,2.5_ds_eeg_bs_16_epochs_15_dpt_0.0_fold0_256,64,16_heads4_08-22-23_fixedpe_PHASE2-TUNE2-fc3_BCE_PEamp_1/lr_0.002_w_1.0,2.5_bs_16_heads4_0.0_att08-22-23_fixedpe_PHASE2-TUNE2-fc3_BCE_epochs15_fold0.pt

CUDA_VISIBLE_DEVICES=1 python trainingLoop.py -lr 0.002 -w 1.0,5 -bs 16 --num_classes 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --simon_model --add_name 08-22-23_fixedpe_PHASE2-TUNE2-fc3_BCE --hidden_size 8 --fc2_size 32 --dropout 0.5 --task binary --pe_fixed --tuning --dropout 0 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_16_epochs_30_dpt_0.5_fold0_256,64,16_heads4_PARAM_TUNER_08-24-23_PEamp_1/lr_0.0002_w_1.0,1.0_bs_16_heads4_0.5_attPARAM_TUNER_08-24-23_epochs30_fold0.pt &

Now a test run with the conv model:
CUDA_VISIBLE_DEVICES=1 python trainingLoop.py -lr 0.0002 -w 1.0,1.0 -bs 64 --num_classes 1 --dataset shhs2 --label antidep --num_epochs 15 --bottleneck_model --add_name 08-28-23_CONV_rerun --task binary --kernel_size 3 --dropout 0.3 --num_heads 4 --stride 1

first going to try to get rid of the conv layers, then going to try adding onother attention layer

Actually since the stride is 1 and the kernel size is 3, really all it is doing is dimensionality reduction on the 768 so I'm going to keep it for now. Down to 128

Seems like adding a layer norm after the attention layer, and adding some dropout to the multi head attn helped

The 4 heads seems to be consisently better. 

Now considering the dimensionality reduction of the bottleneck as well as adding a second multihead attn layer 

Reduction of bottleneck did not help. but two models need further tuning: PE fixed and 2nd attn layer. 2nd attn layer didn't help

Best Run so Far:
CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.0002 -w 1.0,1.0 -bs 64 --num_classes 1 --dataset shhs2 --label antidep --num_epochs 15 --bottleneck_model --add_name 08-28-23_BOTTLENECK_layernormed_attndp01_768256128 --task binary --kernel_size 3 --dropout 0.3 --stride 1 --num_heads 4



In [ ]:
class BottleNeckModel_Antidep(nn.Module):
    def __init__(self, args):
        super(BottleNeckModel_Antidep, self).__init__()
        Scale = 0.5
        args.stride = 1; args.kernel_size=3; args.dropout = 0.3; args.num_classes=1; args.num_heads=4
        # in_channels, out_channels, stride, kernel_size, Scale
        self.attn = nn.MultiheadAttention(768, args.num_heads, dropout=0.1)
        self.block1 = BottleNeck1d(768, 256, args.stride, args.kernel_size, Scale) 
        self.block2 = BottleNeck1d(256, 128, args.stride, args.kernel_size, Scale) 
        # self.block3 = BottleNeck1d(256, 128, args.stride, args.kernel_size, Scale) 
        self.block3=nn.Identity()
        self.relu = nn.ReLU()
        self.norm = nn.LayerNorm(768)
        # self.batch_norm = nn.BatchNorm1d(768)
        # self.pool = nn.AvgPool1d(kernel_size=3)
        
        self.fc1 = nn.Sequential(nn.Linear(150*128, 1024), nn.LayerNorm(1024), nn.Dropout(args.dropout))
        self.fc2 = nn.Sequential(nn.Linear(1024, 256), nn.LayerNorm(256), nn.Dropout(args.dropout))
        self.fc3 = nn.Linear(256, args.num_classes)
        self.relu = nn.ReLU()
    def forward(self, x):
        x,_ = self.attn(x,x,x)
        # x = x+y
        x = self.norm(x)
        conv1= self.block1(x.transpose(1,2))
        conv2= self.block2(conv1)
        conv3 = conv2.view(x.shape[0],-1)
        # bp()
        # conv3= self.block3(conv2)
        
        encoder_output = conv3
        return self.fc3(self.relu(self.fc2(self.relu(self.fc1(encoder_output)))))

2nd stage tuning:

Taking:
exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_64_epochs_15_dpt_0.3_fold0_256,64,16_heads4_08-28-23_BOTTLENECK_layernormed_attndp01_768256128_FINAL_STAGE1_rerun_PEamp_1_Model_bottleneckmodel_kernelsize_3/lr_0.0002_w_1.0,1.0_bs_64_heads4_0.3_att08-28-23_BOTTLENECK_layernormed_attndp01_768256128_FINAL_STAGE1_rerun_epochs15_fold0_best.pt

CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.002 -w 1.0,5 -bs 64 --num_classes 1 --kernel_size 3 --stride 1 --num_heads 4 --dataset shhs2 --label antidep --num_epochs 15 --bottleneck_model --add_name 08-28-23_FC3_TUNING --task binary --tuning --dropout 0 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_64_epochs_15_dpt_0.3_fold0_256,64,16_heads4_08-28-23_BOTTLENECK_layernormed_attndp01_768256128_FINAL_STAGE1_rerun_PEamp_1_Model_bottleneckmodel_kernelsize_3/lr_0.0002_w_1.0,1.0_bs_64_heads4_0.3_att08-28-23_BOTTLENECK_layernormed_attndp01_768256128_FINAL_STAGE1_rerun_epochs15_fold0_best.pt &

Now for the PCA reduced, going to try a few runs:
CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.0002 -w 1.0,1.0 -bs 64 --num_classes 1 --dataset shhs2 --label antidep --num_epochs 15 --pca_model --PCA_embedding --add_name 083123_32pca --task binary --dropout 0.3 --num_heads 4

The main things we can optimize here are the number of heads , dropout, learning rate, batch size, num epochs. the dimensionality of the PCA too but that is later

trying this to first reduce the feature embeddings, then reduce the time embeddings, then gonna try the other way around, then going to try doing both separately and adding

In [ ]:
class CrazyModel_Antidep(nn.Module):
    def __init__(self, args):
        super(CrazyModel_Antidep, self).__init__()
        Scale = 0.5
        # attn_dropout, dropout, sizes, num_heads1 , num_heads2
        attn_dropout = 0.0 if args.tuning else 0.10
        
        self.attn1 = nn.MultiheadAttention(150, args.num_heads, dropout=attn_dropout)
        self.fc1attn1 = nn.Sequential(nn.Linear(768,1024), nn.LayerNorm(1024), nn.Dropout(args.dropout))
        self.fc2attn1 = nn.Sequential(nn.Linear(1024,32), nn.LayerNorm(32), nn.Dropout(args.dropout))
        
        self.attn2 = nn.MultiheadAttention(32, 4, dropout=attn_dropout)
        self.fc2 = nn.Sequential(nn.Linear(150*32, 256), nn.LayerNorm(256), nn.Dropout(args.dropout))
        self.fc3 = nn.Linear(256, args.num_classes)

        # self.attn3 = nn.MultiheadAttention(150, args.num_heads, dropout=attn_dropout, kdim=32)
        # self.block1 = BottleNeck1d(768, 256, args.stride, args.kernel_size, Scale) 
        # self.block2 = BottleNeck1d(256, 128, args.stride, args.kernel_size, Scale) 
        # self.block3 = BottleNeck1d(256, 128, args.stride, args.kernel_size, Scale) 
        self.relu = nn.ReLU()
        self.norm2 = nn.LayerNorm(150)
        self.norm1 = nn.LayerNorm(32)
        # self.batch_norm = nn.BatchNorm1d(768)
        # self.pool = nn.AvgPool1d(kernel_size=3)
        
        

        self.relu = nn.ReLU()
    def forward(self, x):
        x= x.transpose(1,2)
        x,_ = self.attn1(x,x,x)

        # x,_ = self.attn2(x,x,x)
        # x = x+y
        
        
        x = self.norm2(x)
        x= x.transpose(1,2)
        # x = self.norm1(x)
        x = self.relu(self.fc1attn1(x))
        x = self.relu(self.fc2attn1(x))
        
        x,_ = self.attn2(x,x,x)
        # x= x.transpose(1,2)
        # x = self.norm2(x)
        x = self.norm1(x)
        
        x = x.view(x.shape[0],-1)
        x = self.relu(self.fc2(x))
        return self.fc3(x)

A best run for running the stage model:

CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.0002 -w 1.0,1.0 -bs 32 --num_classes 1 --dataset shhs2 --label antidep --num_epochs 15 --stage_model --add_name stage_best_dsize24_attndp01 --task binary --dropout 0.4 --num_heads 1 &

Seems like the best runs get around 71% auroc, about 15% worse than the mage embeddings

exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_256_epochs_15_dpt_0.5_fold0_256,64,16_heads3_stage_test_PEamp_1_Model_stagemodel_kernelsize_5/lr_0.0002_w_1.0,1.0_bs_256_heads3_0.5_attstage_test_epochs15_fold0_best.pt

Second stage tuning:
CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.002 -w 1.0,5 -bs 64 --num_classes 1 --dataset shhs2 --label antidep --num_epochs 15 --stage_model --add_name 090523_stage_fc3tuning --task binary --tuning --dropout 0 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_256_epochs_15_dpt_0.5_fold0_256,64,16_heads3_stage_test_PEamp_1_Model_stagemodel_kernelsize_5/lr_0.0002_w_1.0,1.0_bs_256_heads3_0.5_attstage_test_epochs15_fold0_best.pt &


In [ ]:
class StageModel(nn.Module):
    def __init__(self, args):
        super(StageModel, self).__init__()
        self.d_size = 32
        self.embedding = nn.Embedding(4, self.d_size)
        self.input_size = 2 * 60 * 10
        fc1_size = self.input_size
        self.pe = PositionalEncoding(self.d_size, max_len=self.input_size)
        self.attn = nn.MultiheadAttention(self.d_size, args.num_heads, 0.1)
        self.fc1 = nn.Sequential(nn.Linear(self.input_size , fc1_size), nn.LayerNorm(fc1_size), nn.Dropout(args.dropout), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(fc1_size, 256), nn.LayerNorm(256), nn.Dropout(args.dropout), nn.ReLU())
        self.fc3 = nn.Linear(256, args.num_classes)
    def forward(self, x):
        x = self.embedding(x).squeeze(-1)
        x = self.pe(x)
        x, _ = self.attn(x, x, x)
        # x shape: B x 1000 x d_size
        bp()
        x = torch.mean(x,dim=2)
        # x shape: B x 1000
        ## instead, try to 
        
        # x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)

going to now try to fix the torch.mean issue with conv layers 
works, now the auroc is up to 81% on the val set and the f1 score 51%

CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.0002 -w 1.0,1.0 -bs 32 --num_classes 1 --dataset shhs2 --label antidep --num_epochs 15 --stage_model --add_name rerun11 --task binary --dropout 0.1 --num_heads 1 &

exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_32_epochs_15_dpt_0.1_fold0_256,64,16_heads1_stage_best_dsize24_conv_bn_nofc2_max_PEamp_1_Model_stagemodel_kernelsize_5/lr_0.0002_w_1.0,1.0_bs_32_heads1_0.1_attstage_best_dsize24_conv_bn_nofc2_max_epochs15_fold0_best.pt

Going to do the second stage tuning now

CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.002 -w 1.0,5 -bs 64 --num_classes 1 --dataset shhs2 --label antidep --num_epochs 15 --stage_model --add_name FIXED_TUNING --task binary --tuning --dropout 0 --num_heads 1 --model_path exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_64_epochs_15_dpt_0.1_fold0_256,64,16_heads1_rerun_BUGFIX_FINAL_SAVED_PEamp_1_Model_stagemodel_kernelsize_5_featuredim_16_numtokenheads_4/lr_0.0002_w_1.0,1.0_bs_64_heads1_0.1_attrerun_BUGFIX_FINAL_SAVED_epochs15_fold0_epoch_10.pt &


In [ ]:
class StageModel(nn.Module):
    def __init__(self, args):
        super(StageModel, self).__init__()
        self.d_size = 24
        self.embedding = nn.Embedding(4, self.d_size)
        self.input_size = 2 * 60 * 10
        fc1_size = 256
        self.pe = PositionalEncoding(self.d_size, max_len=self.input_size)
        self.attn = nn.MultiheadAttention(self.d_size, args.num_heads, 0.1)

        self.norm = nn.LayerNorm(self.d_size)
        self.kernel_size = 9
        self.stride = 4
        
        self.conv1 = nn.Conv1d(self.d_size, self.d_size * 2, self.kernel_size, self.stride, padding=4)
        self.conv2 = nn.Conv1d(self.d_size * 2, self.d_size * 4, self.kernel_size, self.stride, padding=4)
        self.conv3 = nn.Conv1d(self.d_size * 4, self.d_size * 8, self.kernel_size, self.stride, padding=4)
        self.conv4 = nn.Conv1d(self.d_size * 8, self.d_size * 16, self.kernel_size, self.stride, padding=4)
        # self.pool = nn.MaxPool1d(kernel_size=10,stride=10)
        
        self.fc1 = nn.Sequential(nn.Linear(self.d_size * 16 , fc1_size), nn.LayerNorm(fc1_size), nn.Dropout(args.dropout), nn.ReLU())
        # self.fc2 = nn.Sequential(nn.Linear(fc1_size, 256), nn.LayerNorm(256), nn.Dropout(args.dropout), nn.ReLU())
        self.fc3 = nn.Linear(256, args.num_classes)
        self.bn1 = nn.BatchNorm1d(self.d_size * 2)
        self.bn2 = nn.BatchNorm1d(self.d_size * 4)
        self.bn3 = nn.BatchNorm1d(self.d_size * 8)
        self.bn4 = nn.BatchNorm1d(self.d_size * 16)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.embedding(x).squeeze(-1)
        x = self.pe(x)
        x, _ = self.attn(x, x, x)
        x = self.norm(x)
        
        # x = self.pool(x.transpose(1,2))
        # x shape: B x 1000 x d_size
        # bp()
        x = x.transpose(1,2)
        # x = torch.mean(x,dim=1)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x,_ = torch.max(x,dim=2)
        # x shape: B x 1000
        ## instead, try to 
        
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        # x = self.fc2(x)
        return self.fc3(x)

just going to stick to this for now:

exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_32_epochs_10_dpt_0.1_fold0_256,64,16_heads1_FINAL_STAGE1_model_best_PEamp_1_Model_stagemodel_kernelsize_5



New mage improvmenet: i have 4 different linear layers that reduce the dimensionality and i concatenate them 

CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.0002 -w 1.0,1.0 -bs 32 --num_classes 1 --dataset shhs2 --label antidep --num_epochs 15 --mage_model --add_name rerun11 --task binary --dropout 0.1 --num_heads 4 --num_token_heads 4 --feature_dim 16 &

Result:
exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_32_epochs_15_dpt_0.1_fold0_256,64,16_heads4_rerun_4tokenhead_16featuredim_PEamp_1_Model_magemodel_kernelsize_5_featuredim_16_numtokenheads_4

Second stage tuning of the Mage model:
CUDA_VISIBLE_DEVICES=0 python trainingLoop.py -lr 0.002 -w 1.0,5 -bs 64 --num_classes 1 --dataset shhs2 --label antidep --num_epochs 15  --num_heads 4 --num_token_heads 4 --feature_dim 8 --mage_model --add_name TUNING_91323 --task binary --tuning --dropout 0 --model_path /data/scratch/alimirz/2023/SIMON/TENSORBOARD/CONV_MODEL/exp_lr_0.0002_w_1.0,1.0_ds_eeg_bs_48_epochs_15_dpt_0.2_fold0_256,64,16_heads4_MAGE_BUGFIX_FINAL_PEamp_1_Model_magemodel_kernelsize_5_featuredim_8_numtokenheads_4/lr_0.0002_w_1.0,1.0_bs_48_heads4_0.2_attMAGE_BUGFIX_FINAL_epochs15_fold0_epoch_6.pt &

In [ ]:
Best Mage Run
    
python trainingLoop.py -lr 0.0002 -w 1.0,1.0 -bs 48 --num_classes 1 --dataset shhs2 --label antidep --num_epochs 15 --mage_model --add_name MAGE_BUGFIX_attndp015_fc1_64 --task binary --dropout 0.2 --num_heads 4 --num_token_heads 4 --feature_dim 8 &